In [1]:
from codebase.classes_ibis_lvm import ParticlesLVM
from codebase.classes_data import Data
from codebase.ibis import model_phonebook, essl
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_stan_model
from run_ibis_lvm import run_ibis_lvm

from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
from codebase.plot import plot_density, get_post_df, plot_correlations, plot_line
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## 2 factor Sim EZ

In [2]:
log_dir = './log/20210103_152056_after_copy_debug/'

## Create Data

In [7]:
exp_data = load_obj('data', log_dir)

In [8]:
exp_data.raw_data['beta']

array([1. , 0.7, 0.8, 0.5, 0.9, 0.6])

In [418]:
exp_data.raw_data

{'random_seed': 0,
 'N': 100,
 'J': 6,
 'K': 1,
 'alpha': array([-0.53,  0.35, -1.4 , -1.4 , -0.96, -2.33]),
 'beta': array([1. , 0.7, 0.8, 0.5, 0.9, 0.6]),
 'z': array([ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ,  1.86755799,
        -0.97727788,  0.95008842, -0.15135721, -0.10321885,  0.4105985 ,
         0.14404357,  1.45427351,  0.76103773,  0.12167502,  0.44386323,
         0.33367433,  1.49407907, -0.20515826,  0.3130677 , -0.85409574,
        -2.55298982,  0.6536186 ,  0.8644362 , -0.74216502,  2.26975462,
        -1.45436567,  0.04575852, -0.18718385,  1.53277921,  1.46935877,
         0.15494743,  0.37816252, -0.88778575, -1.98079647, -0.34791215,
         0.15634897,  1.23029068,  1.20237985, -0.38732682, -0.30230275,
        -1.04855297, -1.42001794, -1.70627019,  1.9507754 , -0.50965218,
        -0.4380743 , -1.25279536,  0.77749036, -1.61389785, -0.21274028,
        -0.89546656,  0.3869025 , -0.51080514, -1.18063218, -0.02818223,
         0.42833187,  0.06651722,  

## Run HMC

In [18]:
ps = load_obj( 'ps_hmc', log_dir)

In [19]:
np.round(np.mean(np.squeeze(ps['beta']),0),2)

array([0.75, 0.07, 0.79, 0.83, 1.31, 0.85])

In [20]:
param = 'beta'
df = get_post_df(np.squeeze(ps[param]).reshape(1000,6,1)) 
df['source'] = 'hmc'
plot_line(df) 
# &\
# plot_correlations(ibis['jitter_corrs'][param], height = 150)

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.Chart(...)

## Run IBIS

In [51]:
# load existing results

particles = load_obj('particles', log_dir)
jitter_corrs = load_obj('jitter_corrs', log_dir)
t = load_obj('t', log_dir)
print(t)

77


In [52]:
# have to resample particles to get rid of weights
particles.resample_particles()

## run final MCMC step if necessary

In [ ]:
particles.resample_particles_bundles()
particles.jitter(t + 1, exp_data.get_stan_data_upto_t(t + 1))
particles.resample_particles()

## Plot Both results

In [53]:
exp_data.raw_data['beta']

array([1. , 0.7, 0.8, 0.5, 0.9, 0.6])

In [54]:
np.round(np.mean(np.squeeze(particles.particles['beta']),0),2)

array([ 0.07,  0.03,  0.01, -0.03,  0.08, -0.01])

In [55]:
param = 'beta'
df = get_post_df(np.squeeze(ps[param]).reshape(1000,6,1)) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) &\
plot_correlations(jitter_corrs[param], height = 150)

alt.VConcatChart(...)

In [26]:
np.mean(np.squeeze(particles.particles['alpha']),0)

array([ 1.08253876,  0.53490661,  0.00346949, -0.53464265, -0.04503675,
       -0.48660115])

In [27]:
param = 'alpha'
df = get_post_df(np.squeeze(ps[param])) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) &\
plot_correlations(jitter_corrs[param], height = 150)

alt.VConcatChart(...)

In [28]:
exp_data.raw_data['alpha']

array([-0.53,  0.35, -1.4 , -1.4 , -0.96, -2.33])